In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install python-terrier

In [ ]:
! pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git

In [4]:
import pyterrier as pt
pt.init()
from pyterrier_doc2query import Doc2Query
doc2query = Doc2Query('macavaney/doc2query-t5-base-msmarco')


terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [5]:
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
def load_data():
  data = []
  with open("/content/gdrive/MyDrive/MLProject/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
      data.append(row)

  book_index = []
  book_id = []
  book_author = []
  book_name = []
  summary = []
  genre = []
  a = 1
  for i in tqdm(data):
      book_index.append(a)
      a = a+1
      book_id.append(i[0])
      book_name.append(i[2])
      book_author.append(i[3])
      genre.append(i[5])
      summary.append(i[6])
  df = pd.DataFrame({'docno': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                        'Genre': genre, 'Summary': summary})
  def clean_data(df):
    df.isna().sum()
    df = df.drop(df[df['Genre'] == ''].index)
    df = df.drop(df[df['Summary'] == ''].index)
    genres_cleaned = []
    for i in df['Genre']:
        genres_cleaned.append(list(json.loads(i).values()))
    df['Genre'] = genres_cleaned

    def clean_summary(text):
      text = re.sub("\'", "", text)
      text = re.sub("[^a-zA-Z]"," ",text)
      text = ' '.join(text.split())
      text = text.lower()
      return text
    df['text'] = df['Summary'].apply(lambda x: clean_summary(x))
    return df
  df = clean_data(df)
  return df

# load document set
df = load_data()

16559it [00:01, 8745.78it/s] 
100%|██████████| 16559/16559 [00:00<00:00, 372029.70it/s]


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12841 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   docno      12841 non-null  int64 
 1   ID         12841 non-null  object
 2   BookTitle  12841 non-null  object
 3   Author     12841 non-null  object
 4   Genre      12841 non-null  object
 5   Summary    12841 non-null  object
 6   text       12841 non-null  object
dtypes: int64(1), object(6)
memory usage: 802.6+ KB


In [7]:
doc2query.append = True # append querygen to text
preprocessed_data = df.to_dict('records')

In [9]:
indexer = pt.IterDictIndexer('/content/gdrive/MyDrive/MLProject/doc2query_index', fields=['text'])
# pipeline = pt.TransformerPipeline([preprocessed_data >> indexer])
# with tqdm(total=len(preprocessed_data), ncols=80) as pbar:
#     for _ in pipeline:
#         pbar.update(1)
pipeline = doc2query >> indexer
pipeline.index(preprocessed_data)

<org.terrier.querying.IndexRef at 0x7fd7a0bd3600 jclass=org/terrier/querying/IndexRef jself=<LocalRef obj=0x363b570 at 0x7fd7a0b39b30>>

In [25]:
index = pt.IndexFactory.of('/content/gdrive/MyDrive/MLProject/doc2query_index')
searcher = pt.BatchRetrieve(index)
results = searcher.transform("book about camping")
results[['qid','docno']].head()


<ipython-input-25-8a143fe12fc6>:3: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  results = searcher.transform("book about camping")


,qid,docno
0,1,11686
1,1,11196
2,1,3102
3,1,10663
4,1,15537


In [26]:
df[df['docno']==11686]

,docno,ID,BookTitle,Author,Genre,Summary,text
11685,11686,15960045,Fateless,Imre Kertész,[Autobiographical novel],"The novel is about a young Hungarian boy, Gyö...",the novel is about a young hungarian boy gy rg...
